In [1]:
import os
import re


In [2]:
# BAM_MAP_LOCATION = '/gscuser/estorrs/CPTAC3.catalog/MGI.BamMap.dat'
# BAM_MAP_LOCATION = '/gscuser/estorrs/gbm.bobo.bammap.dat'
BAM_MAP_LOCATION = '/gscmnt/gc2508/dinglab/estorrs/ancestry_data/mmrf/sample_to_bam.tsv'

EXECUTION_DIR = '/gscmnt/gc2508/dinglab/estorrs/ancestry_data/mmrf/execution'

In [9]:
BAM_MAP_LOCATION = '/gscuser/estorrs/CPTAC3.catalog/BamMap/MGI.BamMap.dat'
EXECUTION_DIR = '/gscmnt/gc2508/dinglab/estorrs/ancestry/executions/cptac3_lscc'

In [6]:
f = open(BAM_MAP_LOCATION)
sample_data = {}


# identifiers = [r'WXS|WGS', 'hg38', 'BAM', r'tissue_normal|blood_normal', 'CCRCC']
identifiers = [r'WXS|WGS', '.bam', 'LSCC', 'blood_normal']
# identifiers = ['blood_normal']


# filter based on identifiers and get sample name and location
for line in f:
    is_valid = True
    for identifier in identifiers:
        if not re.findall(identifier, line):
            is_valid = False
            break

    if is_valid:
        pieces = line.strip().split('\t')

        sample = pieces[1]
        technology = pieces[3]
        fp = pieces[5]
        
        if sample in sample_data:
            d = sample_data[sample]
            if d['technology'].lower() != 'wxs' and technology.lower() == 'wxs':
                sample_data[sample] = {'technology': technology, 'fp': fp}
        else:
            sample_data[sample] = {'technology': technology, 'fp': fp}
len(sample_data), list(sample_data.items())[:5]

(113,
 [('C3L-00081',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/ec948c00-910b-4c7b-82a7-4d209d377116/5e04faec-58e8-403f-942b-74e8c0053805_gdc_realn.bam'}),
  ('C3L-00415',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/9767a9c2-d36b-4cd8-b3bf-e007b9766cc1/ded35b91-a827-4d01-aba8-765d7ba7e4ac_gdc_realn.bam'}),
  ('C3L-00445',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/c84accfb-8a6a-4727-9b19-3ed0e410622e/671050aa-def8-4a5c-9101-f9e53d9e8fe4_wxs_gdc_realn.bam'}),
  ('C3L-00503',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/24b3b72d-c3cb-4dc9-96d2-3e8cf9230b52/9b1d1a96-4652-4b71-961a-1159746b2e10_gdc_realn.bam'}),
  ('C3L-00568',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/63c3f570-84a2-4548-b619-ed7c70a38485/556af585-1b23-404c-ae5f-9a74ebc5356b_gdc_realn.bam'})])

In [7]:
sample_fp_tups = [(k, d['fp']) for k, d in sample_data.items()]
len(sample_fp_tups), sample_fp_tups[:5]

(113,
 [('C3L-00081',
   '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/ec948c00-910b-4c7b-82a7-4d209d377116/5e04faec-58e8-403f-942b-74e8c0053805_gdc_realn.bam'),
  ('C3L-00415',
   '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/9767a9c2-d36b-4cd8-b3bf-e007b9766cc1/ded35b91-a827-4d01-aba8-765d7ba7e4ac_gdc_realn.bam'),
  ('C3L-00445',
   '/gscmnt/gc2741/ding/CPTAC3-data/GDC_import/data/c84accfb-8a6a-4727-9b19-3ed0e410622e/671050aa-def8-4a5c-9101-f9e53d9e8fe4_wxs_gdc_realn.bam'),
  ('C3L-00503',
   '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/24b3b72d-c3cb-4dc9-96d2-3e8cf9230b52/9b1d1a96-4652-4b71-961a-1159746b2e10_gdc_realn.bam'),
  ('C3L-00568',
   '/gscmnt/gc2619/dinglab_cptac3/GDC_import/data/63c3f570-84a2-4548-b619-ed7c70a38485/556af585-1b23-404c-ae5f-9a74ebc5356b_gdc_realn.bam')])

In [10]:
# write sample to bam location file
sample_to_bam_fp = os.path.join(EXECUTION_DIR, 'sample_to_bam.tsv')
out_f = open(sample_to_bam_fp, 'w')
for sample, fp in sample_fp_tups:
    out_f.write(f'{sample}\t{fp}\n')
out_f.close()

In [11]:
os.mkdir(os.path.join(EXECUTION_DIR, 'bams'))

In [12]:
# write softlink script
softlink_script_fp = os.path.join(EXECUTION_DIR, 'symlink_bams.sh')
output_bam_dir = os.path.join(EXECUTION_DIR, 'bams')
out_f = open(softlink_script_fp, 'w')
for sample, fp in sample_fp_tups:
    out_f.write(f'ln -s {fp} {output_bam_dir}/{sample}.bam\n')
out_f.close()